In [7]:
import os
import sys
import cv2
import numpy as np
import argparse


CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
     '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
     '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
     '新',
     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
     'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
     'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
     'W', 'X', 'Y', 'Z', 'I', 'O', '-'
     ]

def decode(preds, CHARS):
    # greedy decode
    pred_labels = list()
    labels = list()
    for i in range(preds.shape[0]):
        pred = preds[i, :, :]
        pred_label = list()
        for j in range(pred.shape[1]):
            pred_label.append(np.argmax(pred[:, j], axis=0))
        no_repeat_blank_label = list()
        pre_c = pred_label[0]
        for c in pred_label: # dropout repeate label and blank label
            if (pre_c == c) or (c == len(CHARS) - 1):
                if c == len(CHARS) - 1:
                    pre_c = c
                continue
            no_repeat_blank_label.append(c)
            pre_c = c
        pred_labels.append(no_repeat_blank_label)
        
    for i, label in enumerate(pred_labels):
        lb = ""
        for i in label:
            lb += CHARS[i]
        labels.append(lb)
    return labels, pred_labels

In [11]:
import cv2
import numpy as np
import onnxruntime as ort

# Leer y preprocesar imagen
img = cv2.imread('test.jpg')
img = cv2.resize(img, (94, 24))

# Convertir a float32 y normalizar
img = img.astype(np.float32) / 255.0

# Reordenar dimensiones si es necesario (ej: HWC -> CHW)
img = np.transpose(img, (2, 0, 1))  # De (H, W, C) a (C, H, W) si el modelo lo requiere

# Añadir dimensión batch
img = np.expand_dims(img, axis=0)  # De (C, H, W) a (1, C, H, W)

# Crear sesión ONNX y correr inferencia
session = ort.InferenceSession("lprnet.onnx", providers=["CPUExecutionProvider"])
outputs = session.run(None, {"input": img})

labels, pred_labels = decode(outputs[0], CHARS)
print('PlACA: '+ labels[0])

PlACA: 湘F6CL03
